In [10]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [3]:
buildings_with_industry = gpd.read_file("data/buildings_with_industry.geojson")
buildings_with_industry['total_area'] = buildings_with_industry['basement_area'] * buildings_with_industry['storeys_count']
buildings_with_industry

,id,function,is_living,storeys_count,basement_area,block_id,industry,geometry,total_area
0,4669322,yes,False,1,694.450,275,industrial,"MULTIPOLYGON (((56.17115 57.88024, 56.17135 57...",694.450
1,79882759,yes,False,1,214.784,752,industrial,"MULTIPOLYGON (((56.34866 58.10470, 56.34892 58...",214.784
2,79882837,office,False,2,776.034,752,service,"MULTIPOLYGON (((56.34712 58.08610, 56.34720 58...",1552.068
3,79883190,yes,False,1,189.197,752,industrial,"MULTIPOLYGON (((56.34799 58.10467, 56.34834 58...",189.197
4,89185196,industrial,False,1,450.628,943,industrial,"MULTIPOLYGON (((56.12423 58.03430, 56.12452 58...",450.628
...,...,...,...,...,...,...,...,...,...
8486,1147821565,yes,False,1,1218.829,649,industrial,"MULTIPOLYGON (((56.16802 57.91086, 56.16836 57...",1218.829
8487,1148104875,yes,False,1,107.470,274,service,"MULTIPOLYGON (((56.13393 57.92601, 56.13408 57...",107.470
8488,1148419580,office,False,4,624.885,276,service,"MULTIPOLYGON (((56.16355 57.88379, 56.16394 57...",2499.540
8489,1148746841,industrial,False,1,188.391,274,industrial,"MULTIPOLYGON (((56.13846 57.90833, 56.13847 57...",188.391


In [6]:
def default_distr(
    distributed_value,
    probabilities_distribution,
    probabilities_names,
):
    """
    Фунция для расселения населения.

    :param distributed_value: Размер выходной случайной выборки
    :param probabilities_distribution: Плотность распредения исходной выборки
    :param probabilities_names: Исходная коллекция, откуда будет производиться выборка

    :return: Numpy массив со случайно сгенерированными величинами с заданой вероятностью
    """
    rng = np.random.default_rng(seed=0)
    return rng.choice(
        a=probabilities_names,
        size=distributed_value,
        p=probabilities_distribution
    )

In [7]:
# Считаем плотность распределения площадей предприятий города
buildings_with_industry["probabilities_distribution"] = \
    buildings_with_industry["total_area"] / buildings_with_industry["total_area"].sum()

buildings_with_industry

,id,function,is_living,storeys_count,basement_area,block_id,industry,geometry,total_area,probabilities_distribution
0,4669322,yes,False,1,694.450,275,industrial,"MULTIPOLYGON (((56.17115 57.88024, 56.17135 57...",694.450,0.000039
1,79882759,yes,False,1,214.784,752,industrial,"MULTIPOLYGON (((56.34866 58.10470, 56.34892 58...",214.784,0.000012
2,79882837,office,False,2,776.034,752,service,"MULTIPOLYGON (((56.34712 58.08610, 56.34720 58...",1552.068,0.000088
3,79883190,yes,False,1,189.197,752,industrial,"MULTIPOLYGON (((56.34799 58.10467, 56.34834 58...",189.197,0.000011
4,89185196,industrial,False,1,450.628,943,industrial,"MULTIPOLYGON (((56.12423 58.03430, 56.12452 58...",450.628,0.000025
...,...,...,...,...,...,...,...,...,...,...
8486,1147821565,yes,False,1,1218.829,649,industrial,"MULTIPOLYGON (((56.16802 57.91086, 56.16836 57...",1218.829,0.000069
8487,1148104875,yes,False,1,107.470,274,service,"MULTIPOLYGON (((56.13393 57.92601, 56.13408 57...",107.470,0.000006
8488,1148419580,office,False,4,624.885,276,service,"MULTIPOLYGON (((56.16355 57.88379, 56.16394 57...",2499.540,0.000141
8489,1148746841,industrial,False,1,188.391,274,industrial,"MULTIPOLYGON (((56.13846 57.90833, 56.13847 57...",188.391,0.000011


In [9]:
# Распределяем трудовые ресурсы города (414 тысяч человек) по всем зданиям
resources = default_distr(414000, buildings_with_industry["probabilities_distribution"], buildings_with_industry.id)
resources

array([167565550, 102959251,  25209192, ..., 109428061, 164142138,
       162806639], dtype=int64)

In [12]:
# Считаем сколько работников попало в каждое здание
counted_resources = pd.Series(resources, name="id").value_counts().sort_index()
counted_resources

id
1342457        111
1439787       2543
1480077          5
1555504          3
1555505          6
              ... 
1147821565      35
1148104875       2
1148419580      67
1148746841       6
1149034816      18
Name: count, Length: 8375, dtype: int64

In [13]:
# добавляем поле с количеством рабочих
buildings_with_industry = buildings_with_industry.merge(counted_resources, left_on='id', right_index=True, how='left')
buildings_with_industry

,id,function,is_living,storeys_count,basement_area,block_id,industry,geometry,total_area,probabilities_distribution,count
0,4669322,yes,False,1,694.450,275,industrial,"MULTIPOLYGON (((56.17115 57.88024, 56.17135 57...",694.450,0.000039,41.0
1,79882759,yes,False,1,214.784,752,industrial,"MULTIPOLYGON (((56.34866 58.10470, 56.34892 58...",214.784,0.000012,9.0
2,79882837,office,False,2,776.034,752,service,"MULTIPOLYGON (((56.34712 58.08610, 56.34720 58...",1552.068,0.000088,72.0
3,79883190,yes,False,1,189.197,752,industrial,"MULTIPOLYGON (((56.34799 58.10467, 56.34834 58...",189.197,0.000011,8.0
4,89185196,industrial,False,1,450.628,943,industrial,"MULTIPOLYGON (((56.12423 58.03430, 56.12452 58...",450.628,0.000025,19.0
...,...,...,...,...,...,...,...,...,...,...,...
8486,1147821565,yes,False,1,1218.829,649,industrial,"MULTIPOLYGON (((56.16802 57.91086, 56.16836 57...",1218.829,0.000069,35.0
8487,1148104875,yes,False,1,107.470,274,service,"MULTIPOLYGON (((56.13393 57.92601, 56.13408 57...",107.470,0.000006,2.0
8488,1148419580,office,False,4,624.885,276,service,"MULTIPOLYGON (((56.16355 57.88379, 56.16394 57...",2499.540,0.000141,67.0
8489,1148746841,industrial,False,1,188.391,274,industrial,"MULTIPOLYGON (((56.13846 57.90833, 56.13847 57...",188.391,0.000011,6.0


In [14]:
buildings_with_industry = buildings_with_industry.drop('probabilities_distribution', axis=1)
buildings_with_industry

,id,function,is_living,storeys_count,basement_area,block_id,industry,geometry,total_area,count
0,4669322,yes,False,1,694.450,275,industrial,"MULTIPOLYGON (((56.17115 57.88024, 56.17135 57...",694.450,41.0
1,79882759,yes,False,1,214.784,752,industrial,"MULTIPOLYGON (((56.34866 58.10470, 56.34892 58...",214.784,9.0
2,79882837,office,False,2,776.034,752,service,"MULTIPOLYGON (((56.34712 58.08610, 56.34720 58...",1552.068,72.0
3,79883190,yes,False,1,189.197,752,industrial,"MULTIPOLYGON (((56.34799 58.10467, 56.34834 58...",189.197,8.0
4,89185196,industrial,False,1,450.628,943,industrial,"MULTIPOLYGON (((56.12423 58.03430, 56.12452 58...",450.628,19.0
...,...,...,...,...,...,...,...,...,...,...
8486,1147821565,yes,False,1,1218.829,649,industrial,"MULTIPOLYGON (((56.16802 57.91086, 56.16836 57...",1218.829,35.0
8487,1148104875,yes,False,1,107.470,274,service,"MULTIPOLYGON (((56.13393 57.92601, 56.13408 57...",107.470,2.0
8488,1148419580,office,False,4,624.885,276,service,"MULTIPOLYGON (((56.16355 57.88379, 56.16394 57...",2499.540,67.0
8489,1148746841,industrial,False,1,188.391,274,industrial,"MULTIPOLYGON (((56.13846 57.90833, 56.13847 57...",188.391,6.0


In [16]:
buildings_with_industry = buildings_with_industry.rename(columns={'count': 'workers'})
buildings_with_industry

,id,function,is_living,storeys_count,basement_area,block_id,industry,geometry,total_area,workers
0,4669322,yes,False,1,694.450,275,industrial,"MULTIPOLYGON (((56.17115 57.88024, 56.17135 57...",694.450,41.0
1,79882759,yes,False,1,214.784,752,industrial,"MULTIPOLYGON (((56.34866 58.10470, 56.34892 58...",214.784,9.0
2,79882837,office,False,2,776.034,752,service,"MULTIPOLYGON (((56.34712 58.08610, 56.34720 58...",1552.068,72.0
3,79883190,yes,False,1,189.197,752,industrial,"MULTIPOLYGON (((56.34799 58.10467, 56.34834 58...",189.197,8.0
4,89185196,industrial,False,1,450.628,943,industrial,"MULTIPOLYGON (((56.12423 58.03430, 56.12452 58...",450.628,19.0
...,...,...,...,...,...,...,...,...,...,...
8486,1147821565,yes,False,1,1218.829,649,industrial,"MULTIPOLYGON (((56.16802 57.91086, 56.16836 57...",1218.829,35.0
8487,1148104875,yes,False,1,107.470,274,service,"MULTIPOLYGON (((56.13393 57.92601, 56.13408 57...",107.470,2.0
8488,1148419580,office,False,4,624.885,276,service,"MULTIPOLYGON (((56.16355 57.88379, 56.16394 57...",2499.540,67.0
8489,1148746841,industrial,False,1,188.391,274,industrial,"MULTIPOLYGON (((56.13846 57.90833, 56.13847 57...",188.391,6.0


In [ ]:
buildings_with_industry = buildings_with_industry.dropna(subset=['workers'])
buildings_with_industry['workers'] = pd.to_numeric(buildings_with_industry['workers'], errors='coerce').astype('Int64')
buildings_with_industry

In [28]:
buildings_with_industry.to_file('data/buildings_with_workers.geojson', driver='GeoJSON')